In [1]:
import numpy as np
import math
import tensorflow as tf

/home/larry/tensorflow_1p5/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/larry/tensorflow_1p5/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/larry/tensorflow_1p5/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/larry/tensorflow_1p5/env/lib/python3.6/site-packages

In [2]:
batch_size=64
embedding_dimension=5
negative_samples=8

In [3]:
digital_to_word_map={1:"One", 2:"Two", 3:"Three", 4:"Four", 5:"Five" ,6:"Six", 7:"Seven", 8:"Eight", 9:"Nine"}

In [4]:
sentences=[]
for i in range(10000):
    rand_odd_ints=np.random.choice(range(1,10,2),3)
    sentences.append(" ".join(digital_to_word_map[r] for r in rand_odd_ints))
    rand_even_ints=np.random.choice(range(2,10,2),3)
    sentences.append(" ".join(digital_to_word_map[r] for r in rand_even_ints))

In [5]:
sentences[:10]

['Three Seven Five',
 'Six Eight Eight',
 'Seven One Three',
 'Four Eight Two',
 'One Five One',
 'Four Four Eight',
 'Nine Seven Seven',
 'Two Six Four',
 'Five Five One',
 'Six Two Four']

In [7]:
word2index_map={}
index=0
for sent in sentences:
    for word in sent.lower().split():
        #print(word)
        if word not in word2index_map:
            word2index_map[word]=index
            index+=1
#print(word2index_map)            
index2word_map={index:word for word,index in word2index_map.items()}
#print(index2word_map)
vocabulary_size=len(index2word_map)
#print(vocabulary_size)

In [8]:
for word,index in word2index_map.items():
    print(word,index)

three 0
seven 1
five 2
six 3
eight 4
one 5
four 6
two 7
nine 8


In [9]:
index2word_map

{0: 'three',
 1: 'seven',
 2: 'five',
 3: 'six',
 4: 'eight',
 5: 'one',
 6: 'four',
 7: 'two',
 8: 'nine'}

In [10]:
skip_gram_pairs=[]
for sent in sentences:
    tokenized_sent = sent.lower().split()
    #print(tokenized_sent)
    #print(len(tokenized_sent))
    for i in range(1, len(tokenized_sent)-1 ):
        print(i)
        #print(tokenized_sent[i-1])
        #print(tokenized_sent[i+1])
        #print(tokenized_sent[i])
        word_contex_pair=[[word2index_map[tokenized_sent[i-1]],
                           word2index_map[tokenized_sent[i+1]]],
                           word2index_map[tokenized_sent[i]]]
        print(word_contex_pair)
        skip_gram_pairs.append([word_contex_pair[1],
                                word_contex_pair[0][0]])
        skip_gram_pairs.append([word_contex_pair[1],
                                word_contex_pair[0][1]])
        #print(skip_gram_pairs)

1
[[0, 2], 1]
1
[[3, 4], 4]
1
[[1, 0], 5]
1
[[6, 7], 4]
1
[[5, 5], 2]
1
[[6, 4], 6]
1
[[8, 1], 1]
1
[[7, 6], 3]
1
[[2, 5], 2]
1
[[3, 6], 7]
1
[[0, 0], 2]
1
[[6, 7], 3]
1
[[2, 0], 1]
1
[[3, 6], 6]
1
[[2, 1], 0]
1
[[7, 7], 4]
1
[[8, 5], 5]
1
[[6, 4], 6]
1
[[2, 1], 1]
1
[[7, 6], 7]
1
[[5, 2], 8]
1
[[3, 3], 3]
1
[[8, 8], 8]
1
[[6, 6], 4]
1
[[2, 1], 5]
1
[[3, 3], 6]
1
[[2, 0], 1]
1
[[7, 3], 4]
1
[[0, 5], 5]
1
[[3, 3], 7]
1
[[5, 1], 8]
1
[[6, 6], 3]
1
[[1, 1], 1]
1
[[4, 3], 7]
1
[[2, 2], 2]
1
[[3, 7], 7]
1
[[1, 1], 8]
1
[[3, 4], 4]
1
[[2, 8], 2]
1
[[6, 3], 3]
1
[[0, 8], 1]
1
[[4, 6], 6]
1
[[2, 5], 8]
1
[[6, 6], 6]
1
[[1, 5], 5]
1
[[3, 4], 4]
1
[[2, 0], 5]
1
[[7, 7], 4]
1
[[8, 2], 8]
1
[[6, 6], 6]
1
[[2, 5], 8]
1
[[4, 6], 3]
1
[[0, 0], 2]
1
[[3, 7], 6]
1
[[0, 8], 8]
1
[[6, 4], 3]
1
[[5, 2], 0]
1
[[4, 3], 3]
1
[[5, 2], 8]
1
[[3, 7], 6]
1
[[5, 8], 0]
1
[[6, 6], 4]
1
[[1, 8], 1]
1
[[3, 6], 6]
1
[[5, 1], 5]
1
[[4, 7], 4]
1
[[5, 5], 0]
1
[[6, 4], 7]
1
[[2, 1], 5]
1
[[6, 4], 4]
1
[[8, 0], 8]
1
[[7,

1
[[3, 6], 3]
1
[[0, 2], 0]
1
[[3, 3], 7]
1
[[2, 8], 8]
1
[[4, 6], 4]
1
[[1, 2], 8]
1
[[7, 4], 6]
1
[[8, 8], 5]
1
[[3, 6], 7]
1
[[2, 2], 2]
1
[[4, 4], 4]
1
[[1, 2], 1]
1
[[3, 6], 4]
1
[[2, 0], 5]
1
[[6, 3], 7]
1
[[1, 5], 5]
1
[[6, 3], 6]
1
[[2, 8], 0]
1
[[6, 7], 6]
1
[[5, 0], 2]
1
[[6, 6], 3]
1
[[5, 0], 1]
1
[[4, 7], 6]
1
[[1, 5], 8]
1
[[3, 6], 3]
1
[[2, 2], 1]
1
[[6, 6], 7]
1
[[5, 8], 1]
1
[[7, 6], 4]
1
[[0, 8], 0]
1
[[6, 7], 3]
1
[[5, 1], 1]
1
[[4, 6], 4]
1
[[1, 5], 5]
1
[[4, 3], 4]
1
[[5, 1], 0]
1
[[4, 7], 3]
1
[[5, 0], 8]
1
[[7, 3], 4]
1
[[0, 0], 8]
1
[[3, 3], 4]
1
[[5, 0], 5]
1
[[4, 4], 7]
1
[[0, 0], 0]
1
[[3, 4], 7]
1
[[0, 2], 2]
1
[[6, 6], 4]
1
[[8, 2], 5]
1
[[6, 4], 4]
1
[[2, 5], 5]
1
[[4, 7], 4]
1
[[2, 1], 8]
1
[[7, 3], 7]
1
[[0, 5], 5]
1
[[7, 6], 7]
1
[[8, 2], 8]
1
[[7, 7], 4]
1
[[0, 5], 0]
1
[[7, 6], 4]
1
[[8, 2], 8]
1
[[3, 7], 7]
1
[[5, 1], 5]
1
[[4, 3], 6]
1
[[2, 0], 5]
1
[[7, 3], 3]
1
[[2, 5], 1]
1
[[4, 6], 7]
1
[[8, 2], 1]
1
[[4, 7], 4]
1
[[8, 0], 2]
1
[[4, 6], 6]
1
[[2,

[[1, 5], 0]
1
[[6, 7], 3]
1
[[0, 1], 1]
1
[[6, 7], 3]
1
[[1, 2], 1]
1
[[6, 6], 7]
1
[[1, 5], 2]
1
[[6, 7], 7]
1
[[2, 0], 8]
1
[[3, 3], 4]
1
[[1, 1], 5]
1
[[6, 7], 6]
1
[[0, 0], 5]
1
[[3, 3], 3]
1
[[8, 1], 2]
1
[[3, 3], 6]
1
[[5, 1], 5]
1
[[6, 4], 3]
1
[[2, 1], 5]
1
[[4, 7], 4]
1
[[5, 1], 1]
1
[[6, 7], 6]
1
[[1, 8], 5]
1
[[7, 6], 7]
1
[[1, 2], 2]
1
[[6, 3], 7]
1
[[5, 0], 2]
1
[[3, 3], 7]
1
[[8, 1], 2]
1
[[4, 3], 7]
1
[[2, 1], 0]
1
[[3, 6], 3]
1
[[1, 0], 5]
1
[[6, 3], 6]
1
[[1, 1], 0]
1
[[6, 3], 3]
1
[[0, 8], 0]
1
[[6, 4], 6]
1
[[2, 5], 0]
1
[[6, 7], 4]
1
[[2, 5], 8]
1
[[4, 6], 7]
1
[[5, 0], 0]
1
[[4, 4], 7]
1
[[5, 0], 5]
1
[[7, 4], 7]
1
[[2, 5], 5]
1
[[6, 4], 3]
1
[[8, 2], 2]
1
[[4, 6], 3]
1
[[0, 2], 1]
1
[[3, 6], 7]
1
[[1, 2], 0]
1
[[3, 3], 3]
1
[[0, 8], 8]
1
[[3, 3], 4]
1
[[0, 0], 2]
1
[[7, 3], 4]
1
[[1, 2], 0]
1
[[6, 7], 4]
1
[[1, 2], 2]
1
[[7, 4], 4]
1
[[1, 8], 5]
1
[[3, 4], 7]
1
[[1, 0], 8]
1
[[7, 3], 7]
1
[[1, 0], 0]
1
[[7, 4], 6]
1
[[2, 0], 1]
1
[[4, 3], 4]
1
[[0, 5], 8]
1
[[3, 4

[[5, 8], 8]
1
[[6, 6], 3]
1
[[8, 1], 5]
1
[[7, 7], 4]
1
[[2, 5], 2]
1
[[3, 3], 7]
1
[[5, 2], 2]
1
[[6, 4], 7]
1
[[1, 2], 0]
1
[[4, 6], 6]
1
[[2, 1], 8]
1
[[4, 7], 4]
1
[[0, 5], 5]
1
[[4, 4], 7]
1
[[5, 0], 2]
1
[[6, 6], 6]
1
[[2, 8], 1]
1
[[7, 7], 3]
1
[[8, 5], 2]
1
[[3, 7], 7]
1
[[1, 8], 8]
1
[[6, 7], 6]
1
[[0, 0], 8]
1
[[4, 3], 7]
1
[[0, 5], 2]
1
[[4, 4], 4]
1
[[8, 8], 2]
1
[[4, 3], 4]
1
[[5, 5], 8]
1
[[6, 4], 7]
1
[[2, 8], 1]
1
[[6, 4], 7]
1
[[5, 8], 1]
1
[[4, 4], 4]
1
[[1, 2], 1]
1
[[6, 6], 7]
1
[[1, 8], 1]
1
[[6, 7], 6]
1
[[2, 0], 2]
1
[[3, 6], 7]
1
[[5, 0], 2]
1
[[7, 7], 6]
1
[[0, 1], 2]
1
[[4, 7], 6]
1
[[1, 0], 8]
1
[[3, 4], 4]
1
[[5, 2], 1]
1
[[6, 4], 4]
1
[[5, 5], 0]
1
[[6, 3], 6]
1
[[8, 8], 0]
1
[[6, 3], 3]
1
[[8, 1], 1]
1
[[6, 3], 4]
1
[[0, 0], 5]
1
[[6, 7], 7]
1
[[2, 8], 1]
1
[[3, 4], 4]
1
[[8, 1], 0]
1
[[3, 7], 6]
1
[[2, 2], 2]
1
[[7, 3], 7]
1
[[1, 8], 8]
1
[[4, 4], 4]
1
[[1, 2], 1]
1
[[4, 4], 4]
1
[[8, 5], 1]
1
[[6, 4], 3]
1
[[8, 8], 1]
1
[[7, 6], 7]
1
[[2, 8], 1]
1
[[4, 3

1
[[0, 1], 0]
1
[[4, 4], 7]
1
[[2, 5], 0]
1
[[3, 6], 6]
1
[[2, 8], 0]
1
[[7, 7], 4]
1
[[5, 2], 2]
1
[[6, 6], 3]
1
[[5, 1], 2]
1
[[6, 3], 4]
1
[[0, 2], 2]
1
[[4, 6], 3]
1
[[0, 1], 5]
1
[[3, 3], 3]
1
[[1, 5], 5]
1
[[4, 3], 7]
1
[[5, 5], 2]
1
[[3, 7], 4]
1
[[2, 0], 1]
1
[[3, 4], 3]
1
[[2, 0], 0]
1
[[3, 7], 7]
1
[[1, 5], 0]
1
[[3, 3], 7]
1
[[8, 0], 2]
1
[[7, 3], 3]
1
[[8, 0], 5]
1
[[7, 3], 6]
1
[[0, 0], 0]
1
[[7, 3], 6]
1
[[5, 5], 2]
1
[[6, 7], 7]
1
[[8, 0], 8]
1
[[4, 4], 6]
1
[[1, 2], 5]
1
[[4, 6], 3]
1
[[1, 1], 0]
1
[[4, 7], 3]
1
[[1, 2], 1]
1
[[7, 7], 3]
1
[[8, 0], 5]
1
[[6, 7], 7]
1
[[8, 0], 8]
1
[[3, 3], 6]
1
[[1, 8], 2]
1
[[3, 6], 3]
1
[[8, 5], 0]
1
[[3, 4], 3]
1
[[2, 0], 2]
1
[[6, 6], 3]
1
[[2, 5], 2]
1
[[7, 4], 3]
1
[[0, 2], 2]
1
[[6, 3], 6]
1
[[0, 0], 0]
1
[[7, 4], 4]
1
[[0, 1], 5]
1
[[4, 4], 7]
1
[[8, 1], 2]
1
[[3, 4], 3]
1
[[0, 2], 8]
1
[[7, 6], 3]
1
[[8, 2], 0]
1
[[6, 6], 6]
1
[[5, 5], 8]
1
[[4, 6], 3]
1
[[5, 2], 5]
1
[[7, 7], 4]
1
[[0, 5], 8]
1
[[4, 7], 4]
1
[[1, 2], 1]
1
[[4,

...

[[1, 2], 2]
1
[[4, 4], 7]
1
[[2, 8], 5]
1
[[7, 7], 4]
1
[[2, 8], 1]
1
[[7, 4], 4]
1
[[8, 5], 1]
1
[[3, 7], 4]
1
[[1, 0], 1]
1
[[3, 4], 3]
1
[[2, 5], 2]
1
[[7, 4], 6]
1
[[0, 2], 5]
1
[[4, 6], 6]
1
[[1, 0], 1]
1
[[3, 7], 3]
1
[[2, 0], 2]
1
[[6, 6], 3]
1
[[1, 8], 1]
1
[[4, 7], 7]
1
[[1, 8], 5]
1
[[7, 6], 4]
1
[[2, 1], 8]
1
[[7, 7], 4]
1
[[1, 5], 5]
1
[[6, 6], 3]
1
[[1, 8], 8]
1
[[4, 4], 3]
1
[[5, 5], 1]
1
[[7, 4], 6]
1
[[8, 1], 2]
1
[[7, 4], 7]
1
[[2, 8], 2]
1
[[4, 6], 7]
1
[[0, 1], 5]
1
[[4, 6], 6]
1
[[1, 8], 0]
1
[[4, 4], 4]
1
[[5, 2], 2]
1
[[4, 6], 4]
1
[[0, 1], 5]
1
[[3, 7], 4]
1
[[8, 1], 0]
1
[[3, 6], 7]
1
[[2, 1], 2]
1
[[3, 7], 7]
1
[[1, 0], 0]
1
[[3, 3], 4]
1
[[0, 0], 1]
1
[[3, 4], 3]
1
[[8, 5], 2]
1
[[7, 3], 7]
1
[[2, 8], 2]
1
[[3, 6], 4]
1
[[2, 1], 0]
1
[[7, 7], 3]
1
[[0, 0], 2]
1
[[6, 7], 6]
1
[[8, 2], 8]
1
[[4, 3], 6]
1
[[8, 8], 8]
1
[[3, 3], 4]
1
[[8, 1], 2]
1
[[3, 7], 6]
1
[[5, 5], 8]
1
[[6, 6], 4]
1
[[5, 1], 0]
1
[[7, 3], 4]
1
[[5, 0], 5]
1
[[7, 7], 3]
1
[[8, 2], 1]
1
[[7, 7

1
[[3, 6], 7]
1
[[5, 2], 1]
1
[[3, 3], 7]
1
[[2, 0], 1]
1
[[7, 7], 3]
1
[[5, 2], 5]
1
[[3, 7], 3]
1
[[0, 5], 1]
1
[[7, 7], 3]
1
[[8, 2], 0]
1
[[4, 3], 3]
1
[[0, 5], 2]
1
[[4, 3], 3]
1
[[8, 2], 2]
1
[[3, 3], 4]
1
[[1, 8], 5]
1
[[6, 3], 7]
1
[[0, 0], 0]
1
[[7, 7], 6]
1
[[5, 1], 1]
1
[[4, 4], 4]
1
[[8, 1], 8]
1
[[6, 6], 4]
1
[[8, 5], 1]
1
[[7, 4], 3]
1
[[0, 8], 8]
1
[[3, 3], 7]
1
[[0, 0], 5]
1
[[4, 4], 3]
1
[[1, 1], 8]
1
[[3, 3], 7]
1
[[2, 2], 2]
1
[[4, 3], 3]
1
[[0, 2], 0]
1
[[6, 4], 4]
1
[[8, 2], 2]
1
[[7, 4], 7]
1
[[5, 0], 5]
1
[[6, 3], 4]
1
[[0, 1], 0]
1
[[7, 6], 6]
1
[[1, 2], 5]
1
[[4, 6], 7]
1
[[8, 5], 2]
1
[[4, 3], 4]
1
[[1, 0], 1]
1
[[3, 3], 7]
1
[[8, 1], 1]
1
[[7, 4], 6]
1
[[1, 1], 5]
1
[[7, 7], 7]
1
[[2, 1], 2]
1
[[6, 6], 6]
1
[[1, 8], 2]
1
[[6, 3], 7]
1
[[1, 5], 1]
1
[[4, 4], 4]
1
[[8, 0], 1]
1
[[4, 6], 7]
1
[[2, 1], 2]
1
[[6, 4], 3]
1
[[1, 5], 2]
1
[[3, 3], 7]
1
[[1, 0], 5]
1
[[4, 6], 3]
1
[[1, 8], 8]
1
[[7, 3], 3]
1
[[1, 0], 1]
1
[[7, 7], 3]
1
[[2, 0], 5]
1
[[4, 6], 7]
1
[[1,

[[4, 3], 6]
1
[[8, 1], 1]
1
[[7, 4], 3]
1
[[5, 2], 2]
1
[[7, 4], 7]
1
[[2, 0], 5]
1
[[7, 6], 4]
1
[[5, 5], 2]
1
[[7, 3], 7]
1
[[1, 1], 8]
1
[[3, 6], 6]
1
[[2, 1], 0]
1
[[3, 7], 3]
1
[[2, 1], 2]
1
[[7, 6], 3]
1
[[2, 1], 2]
1
[[4, 6], 7]
1
[[0, 2], 1]
1
[[7, 4], 6]
1
[[5, 5], 1]
1
[[4, 3], 4]
1
[[1, 0], 1]
1
[[7, 4], 3]
1
[[5, 8], 8]
1
[[6, 6], 4]
1
[[0, 5], 1]
1
[[3, 7], 6]
1
[[2, 5], 8]
1
[[3, 7], 6]
1
[[2, 5], 1]
1
[[4, 3], 4]
1
[[2, 2], 1]
1
[[7, 6], 4]
1
[[2, 0], 8]
1
[[3, 4], 3]
1
[[5, 0], 8]
1
[[6, 4], 4]
1
[[8, 5], 5]
1
[[7, 7], 3]
1
[[2, 0], 2]
1
[[6, 6], 3]
1
[[5, 1], 0]
1
[[3, 3], 6]
1
[[2, 1], 5]
1
[[6, 3], 7]
1
[[8, 5], 2]
1
[[4, 6], 3]
1
[[8, 8], 2]
1
[[6, 3], 7]
1
[[2, 5], 2]
1
[[4, 6], 3]
1
[[5, 1], 0]
1
[[4, 3], 4]
1
[[1, 2], 2]
1
[[6, 3], 7]
1
[[1, 0], 2]
1
[[6, 3], 7]
1
[[0, 0], 1]
1
[[7, 3], 6]
1
[[1, 8], 2]
1
[[3, 7], 3]
1
[[1, 2], 0]
1
[[6, 6], 6]
1
[[1, 0], 8]
1
[[4, 7], 6]
1
[[2, 1], 8]
1
[[6, 7], 3]
1
[[8, 8], 0]
1
[[3, 7], 7]
1
[[2, 5], 8]
1
[[3, 7], 7]
1
[[0, 1

[[7, 7], 4]
1
[[8, 8], 8]
1
[[3, 4], 4]
1
[[2, 8], 2]
1
[[6, 6], 3]
1
[[2, 8], 2]
1
[[7, 3], 3]
1
[[5, 0], 1]
1
[[6, 4], 4]
1
[[2, 2], 0]
1
[[3, 4], 6]
1
[[1, 8], 5]
1
[[6, 6], 4]
1
[[0, 8], 0]
1
[[3, 7], 3]
1
[[2, 1], 2]
1
[[7, 7], 4]
1
[[5, 5], 8]
1
[[4, 7], 7]
1
[[1, 5], 2]
1
[[4, 3], 4]
1
[[5, 5], 2]
1
[[6, 7], 6]
1
[[2, 8], 5]
1
[[4, 6], 4]
1
[[0, 1], 5]
1
[[3, 3], 6]
1
[[8, 2], 0]
1
[[7, 6], 3]
1
[[1, 8], 2]
1
[[3, 6], 4]
1
[[8, 1], 5]
1
[[4, 7], 6]
1
[[8, 8], 0]
1
[[7, 4], 4]
1
[[8, 5], 5]
1
[[3, 7], 4]
1
[[0, 8], 8]
1
[[7, 3], 4]
1
[[2, 0], 1]
1
[[4, 3], 4]
1
[[8, 5], 2]
1
[[4, 4], 6]
1
[[1, 1], 2]
1
[[4, 3], 6]
1
[[8, 1], 0]
1
[[6, 3], 4]
1
[[0, 2], 1]
1
[[3, 6], 6]
1
[[0, 0], 8]
1
[[6, 6], 7]
1
[[5, 0], 5]
1
[[4, 6], 7]
1
[[5, 2], 2]
1
[[4, 3], 7]
1
[[1, 8], 0]
1
[[3, 7], 4]
1
[[0, 8], 1]
1
[[6, 4], 4]
1
[[0, 5], 8]
1
[[4, 3], 4]
1
[[8, 0], 2]
1
[[3, 6], 6]
1
[[5, 2], 5]
1
[[3, 3], 6]
1
[[5, 2], 1]
1
[[6, 7], 7]
1
[[1, 1], 0]
1
[[3, 4], 4]
1
[[5, 1], 1]
1
[[7, 7], 4]
1
[[2, 8

[[7, 3], 6]
1
[[2, 2], 2]
1
[[4, 3], 4]
1
[[2, 1], 1]
1
[[4, 4], 4]
1
[[5, 0], 1]
1
[[6, 4], 4]
1
[[0, 8], 2]
1
[[4, 4], 3]
1
[[1, 0], 2]
1
[[6, 3], 4]
1
[[2, 0], 5]
1
[[3, 3], 6]
1
[[5, 0], 8]
1
[[6, 6], 4]
1
[[8, 2], 0]
1
[[6, 6], 3]
1
[[5, 1], 2]
1
[[4, 6], 6]
1
[[2, 0], 0]
1
[[3, 7], 7]
1
[[2, 2], 0]
1
[[7, 7], 4]
1
[[2, 2], 5]
1
[[6, 7], 4]
1
[[2, 2], 0]
1
[[6, 4], 7]
1
[[2, 2], 2]
1
[[3, 6], 4]
1
[[0, 0], 0]
1
[[4, 6], 7]
1
[[2, 5], 0]
1
[[6, 3], 4]
1
[[0, 1], 8]
1
[[6, 7], 4]
1
[[1, 0], 0]
1
[[3, 7], 3]
1
[[1, 8], 2]
1
[[3, 7], 6]
1
[[0, 2], 8]
1
[[4, 7], 4]
1
[[8, 2], 1]
1
[[3, 7], 3]
1
[[5, 8], 0]
1
[[6, 4], 3]
1
[[2, 2], 8]
1
[[4, 6], 7]
1
[[1, 1], 8]
1
[[3, 3], 6]
1
[[0, 5], 0]
1
[[7, 3], 7]
1
[[8, 2], 2]
1
[[6, 4], 7]
1
[[1, 1], 8]
1
[[4, 4], 7]
1
[[0, 2], 0]
1
[[4, 3], 3]
1
[[5, 1], 8]
1
[[7, 7], 7]
1
[[2, 2], 1]
1
[[3, 6], 3]
1
[[0, 1], 1]
1
[[6, 4], 3]
1
[[5, 5], 0]
1
[[3, 6], 4]
1
[[2, 2], 8]
1
[[7, 4], 4]
1
[[1, 1], 1]
1
[[4, 4], 6]
1
[[0, 1], 5]
1
[[3, 3], 3]
1
[[5, 0

In [13]:
def get_skipgram_batch(batch_size):
    instant_indices = list(range(len(skip_gram_pairs)))
    np.random.shuffle(instant_indices)
    batch=instant_indices[:batch_size]
    x=[skip_gram_pairs[i][0] for i in batch]
    y=[[skip_gram_pairs[i][1]] for i in batch]
    return x,y

In [14]:
x_batch,y_batch=get_skipgram_batch(8)

In [15]:
x_batch

[0, 0, 1, 1, 7, 7, 6, 4]

In [16]:
y_batch

[[1], [2], [0], [5], [4], [6], [3], [7]]

In [17]:
[index2word_map[word] for word in x_batch]

['three', 'three', 'seven', 'seven', 'two', 'two', 'four', 'eight']

In [18]:
[index2word_map[word[0]] for word in y_batch ] 

['seven', 'five', 'three', 'one', 'eight', 'four', 'six', 'two']

In [19]:
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])

In [20]:
train_labels=tf.placeholder(tf.int32, shape=[batch_size,1])

In [21]:
embedding=tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_dimension],-1.0,1.0)
    ,name='embedding'
)

In [22]:
embed=tf.nn.embedding_lookup(embedding, train_inputs)

In [23]:
nce_weights=tf.Variable(tf.truncated_normal([vocabulary_size, embedding_dimension],
                                           stddev=1.0/math.sqrt(embedding_dimension)))

In [24]:
nce_biases=tf.Variable(tf.zeros([vocabulary_size]))

In [25]:
loss=tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, inputs=embed,
                  labels=train_labels, num_sampled=negative_samples, 
                  num_classes=vocabulary_size)
)

In [26]:
global_step=tf.Variable(0, trainable=False)
learningRate=tf.train.exponential_decay(learning_rate=0.1,
                                       global_step=global_step,
                                       decay_steps=1000,
                                       decay_rate=0.95,
                                       staircase=True)

In [27]:
train_step=tf.train.GradientDescentOptimizer(learningRate).minimize(loss)

In [28]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for step in range(1000):
        x_batch,y_batch=get_skipgram_batch(batch_size)
        sess.run(train_step,feed_dict={train_inputs:x_batch,train_labels:y_batch})
        if step % 100 ==0:
            loss_val=sess.run(loss,feed_dict={train_inputs:x_batch,train_labels:y_batch})
            print(loss_val)
    norm=tf.sqrt(tf.reduce_sum(tf.square(embedding),1,keep_dims=True))
    normalized_embedding = embedding/norm
    normalized_embedding_matrix=sess.run(normalized_embedding)

6.673084
3.1299446
2.9821491
2.698881
2.5955768
2.5292816
2.570719
2.5264776
2.4903107
2.5354323
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [29]:
normalized_embedding_matrix

array([[ 0.351201  , -0.3523663 ,  0.4550719 , -0.574611  , -0.46392632],
       [-0.03102761, -0.28331646,  0.481048  , -0.7173584 , -0.41564265],
       [-0.35620746,  0.37718233,  0.7826011 , -0.3432614 , -0.02359535],
       [ 0.25911084, -0.27397633,  0.12416907, -0.03465624,  0.9171584 ],
       [ 0.10235577,  0.18474622, -0.27680284, -0.46753922,  0.81251425],
       [ 0.19686015,  0.31659475,  0.6205582 , -0.627912  , -0.28574112],
       [ 0.2341139 , -0.28012812,  0.17857453,  0.02342678,  0.91339   ],
       [ 0.15852429,  0.01629568, -0.2159582 , -0.37230608,  0.88845646],
       [-0.04814674,  0.19286701,  0.6873348 , -0.6357862 , -0.2895361 ]],
      dtype=float32)

In [30]:
word2index_map["one"]

5

In [31]:
ref_word=normalized_embedding_matrix[word2index_map["one"]]

In [32]:
ref_word

array([ 0.19686015,  0.31659475,  0.6205582 , -0.627912  , -0.28574112],
      dtype=float32)

In [39]:
ref_word.shape

(5,)

In [38]:
normalized_embedding_matrix.shape

(9, 5)

In [33]:
cosine_digits = np.dot(normalized_embedding_matrix, ref_word)

In [34]:
cosine_digits 

array([ 0.7333467 ,  0.7719178 ,  0.75722057, -0.19898555, -0.03172809,
        1.0000001 , -0.20748653, -0.11774148,  0.9600639 ], dtype=float32)

In [40]:
cosine_digits.shape

(9,)

In [41]:
ff=np.argsort(cosine_digits )[::-1][1:10]

In [42]:
ff

array([8, 1, 2, 0, 4, 7, 3, 6])

In [45]:
index2word_map

{0: 'three',
 1: 'seven',
 2: 'five',
 3: 'six',
 4: 'eight',
 5: 'one',
 6: 'four',
 7: 'two',
 8: 'nine'}

In [44]:
for f in ff:
    print(index2word_map[f])
    print(cosine_digits[f])
    print('  ')

nine
0.9600639
  
seven
0.7719178
  
five
0.75722057
  
three
0.7333467
  
eight
-0.03172809
  
two
-0.11774148
  
six
-0.19898555
  
four
-0.20748653
  
